<img src="https://s2.glbimg.com/Bu6upvmSg6SRv0za635uXphThKo=/620x430/e.glbimg.com/og/ed/f/original/2020/03/28/mercado-livre.jpg" width=30%/>

# Data Science Challenge - Data & Analytics Team
## 2. Similitud entre productos

### Descripción

Un desafío constante en MELI es el de poder agrupar productos similares utilizando algunos atributos de estos como pueden ser el título, la descripción o su imagen.

Para este desafío tenemos un dataset “items_titles.csv” que tiene títulos de 30 mil productos de 3 categorías diferentes de Mercado Libre Brasil.

El objetivo del desafío es poder generar una Jupyter notebook que determine cuán similares son dos títulos del dataset “item_titles_test.csv” generando como output un listado de la forma...

Donde ordenando por score de similitud podamos encontrar los pares de productos más similares en nuestro dataset de test.

In [ ]:
# Instalação do Sentence Transformers (https://www.sbert.net/index.html)
# pip install -U sentence-transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:
# Armazenamento de csvs em variáveis
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MELI/items_titles.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MELI/items_titles_test.csv')

In [5]:
print(train.shape)
print(test.shape)

(30000, 1)
(10000, 1)


In [6]:
train.head()

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...


In [7]:
test.head()

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde


In [8]:
# Convertendo colunas dos dois datasets para lowercase
train['ITE_ITEM_TITLE'] = train['ITE_ITEM_TITLE'].str.lower()
test['ITE_ITEM_TITLE'] = test['ITE_ITEM_TITLE'].str.lower()

Decidi não remover pontuação e caracteres especiais pois os títulos são criados pelos vendedores.

In [38]:
# Importando SentenceTransformer
from sentence_transformers import SentenceTransformer

# Instanciando o modelo pré-treinado all-MiniLM-L6-v2 do Hugging Face (https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [27]:
# Importando Cosine Similarity do Scikit-learn
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
# Encoding de valores do dataset train usando o model (all-MiniLM-L6-v2)
train_vecs = model.encode(train['ITE_ITEM_TITLE'][:3000])

In [29]:
train_vecs.shape

(3000, 384)

Temos um array com 3.000 linhas e 384 de tamanho no hidden state size do modelo Bert.

In [30]:
train_vecs

array([[-1.04104668e-01,  8.30359012e-02, -1.26138926e-04, ...,
         9.03577879e-02, -2.26802342e-02, -5.28213456e-02],
       [-4.46924120e-02,  4.16178182e-02, -1.40467435e-01, ...,
         5.14654107e-02, -3.91320102e-02,  8.16617440e-03],
       [-8.95926803e-02,  6.88367337e-02, -3.57197761e-03, ...,
         7.81936944e-02, -3.78785692e-02,  4.12488764e-04],
       ...,
       [-8.23145453e-03,  5.90097858e-03,  3.20991650e-02, ...,
         5.90163618e-02,  9.36579797e-03, -5.74092939e-02],
       [-8.66851658e-02, -2.38280445e-02,  7.22426176e-02, ...,
         2.30916915e-03,  2.22372683e-03,  2.08990020e-03],
       [-9.95980389e-03,  1.16068095e-01, -9.98413712e-02, ...,
         9.19225514e-02,  8.36134106e-02, -2.55244002e-02]], dtype=float32)

In [31]:
# Encoding de valores do dataset test usando o model (all-MiniLM-L6-v2)
test_vecs = model.encode(test['ITE_ITEM_TITLE'][:1000])

In [32]:
test_vecs.shape

(1000, 384)

Temos um array com 1000 linhas e 384 de tamanho no hidden state size do modelo Bert.

In [33]:
test_vecs

array([[-0.01816418,  0.12168916, -0.00326645, ...,  0.08843251,
         0.08487038, -0.03037256],
       [-0.04601841,  0.07405472, -0.1056786 , ..., -0.04466264,
        -0.00369643, -0.01232435],
       [-0.08496193, -0.06072852,  0.05279116, ..., -0.00343313,
        -0.00308601, -0.06607358],
       ...,
       [ 0.00535024, -0.02248265, -0.03684547, ..., -0.00135909,
         0.017242  ,  0.00710275],
       [-0.14282513,  0.07787596, -0.04585041, ..., -0.05926555,
        -0.07173265,  0.03149602],
       [-0.0677074 ,  0.03779507, -0.00163527, ...,  0.01686839,
         0.02153157, -0.01346844]], dtype=float32)

In [34]:
# Importando tqdm para acompanhar progresso do loop
from tqdm import tqdm

# Criação de listas vazias para armazenar variáveis no loop
word_train = []
word_test = []
word_similarity = []
original_train = []
original_test = []

''' 
Loop rodando nos dois datasets e aplicando cosine_similarity (Testando 3.000 
itens no dataset train com cada um de 1.000 do dataset de test).
O Colab não aguentou o processamento dos dois datasets inteiros pois geraria um
dataframe com mais de 300 milhões de linhas.
'''

for i in tqdm(range(0,3000)):
  for j in range(0, 1000):
    cosine_sim = cosine_similarity(
      train_vecs[i].reshape(1, -1),
      test_vecs[j].reshape(1, -1)
    )
  
    # Scaling de similaridade para ficar entre 0 e 1
    scaled_sim = 0.5 * (cosine_sim + 1)

    # Salvando itens nas listas
    word_train.append(train_vecs[i])
    word_test.append(test_vecs[j])
    word_similarity.append(scaled_sim)
    original_train.append(train['ITE_ITEM_TITLE'][i])
    original_test.append(test['ITE_ITEM_TITLE'][j])

100%|██████████| 3000/3000 [17:20<00:00,  2.88it/s]


In [35]:
# Criação do dataframe
df = pd.DataFrame({
    'ITE_ITEM_TITLE':original_train,
    'ITE_ITEM_TITLE2':original_test,
    'Score Similitud (0,1)':word_similarity
})

In [42]:
# Visualização do dataframe ordenado de maneira decrescente pela similaridade
df = df.sort_values(by='Score Similitud (0,1)', ascending=False)
display(df)

,ITE_ITEM_TITLE,ITE_ITEM_TITLE2,"Score Similitud (0,1)"
1541718,tenis fila 969532 trend 2.0 /vermelho 969532,tenis fila 969532 trend 2.0 /vermelho 969532,[[1.0000001]]
2793088,tênis infantil menina barbie calça fácil s/ ca...,tênis infantil menina barbie calça fácil s/ ca...,[[1.0]]
2699694,tênis esportivo rainha play original,tênis esportivo rainha play original,[[1.0]]
2627912,tenis feminino guide / 590 - olympikus (04) - ...,tenis feminino guide / 590 - olympikus (04) - ...,[[1.0]]
787819,tenis masculino casual oldsen academia esporte...,tenis masculino casual oldsen academia esporte...,[[1.0]]
...,...,...,...
1157696,smart tv philco ptv32g70sbl led hd 32 110v/220v,tênis fuzex workout training crossfit wod,[[0.40851957]]
1537093,bike mtb rava pressure 29,sapatos esportivos femininos com plataforma le...,[[0.40736347]]
591654,cannondale fsi carbon 4 2020,sapatênis masculino slipe on adulto pomoção luxo,[[0.40712065]]
1779013,smart tv hq hqstv43ny led 4k 43 110v/220v,tenis ous emergente copper essencial,[[0.4070005]]


Dataframe criado e ordenado por similaridade usando Cosine Similarity.

In [43]:
# Salvando resultado como csv
df.to_csv('meli2_results.csv')

## Obrigado.

**Felipe Nobre**<br />
*Aspirante a Cientista de Dados no MELI*<br /><br />
Telefone: +55 19 99909-9510<br />
E-mail: campostqe@gmail.com<br />
LinkedIn: <a href="https://www.linkedin.com/in/felipenobrecampos/">https://www.linkedin.com/in/felipenobrecampos/</a>